In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.compose import EnsembleForecaster

#Online Expert Files
from sktime.forecasting.ensemble_algorithms import HedgeExpertEnsemble, NNLSEnsemble
from sktime.forecasting.online_experts import NormalHedge, OnlineHedgeDoubling, se
from sktime.forecasting.online_ensemble import OnlineEnsembleForecaster

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sktime.datasets import load_airline
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.performance_metrics.forecasting import smape_loss
from sktime.utils.plotting.forecasting import plot_ys

%matplotlib inline

# Airline Dataset Problem Setup

In [ ]:
y = load_airline()
fig, ax = plot_ys(y)
ax.set(xlabel="Time", ylabel="Number of airline passengers");

In [ ]:
y_train, y_test = temporal_train_test_split(y, test_size=36)
print(y_train.shape[0], y_test.shape[0])

In [ ]:
fh = np.arange(len(y_test)) + 1
fh

# Ensemble Algorithms

## Averaging Ensemble

In [ ]:
forecaster = EnsembleForecaster([
    ("ses", ExponentialSmoothing(seasonal="multiplicative", sp=12)),
    ("holt", ExponentialSmoothing(trend="add", damped=False, seasonal="multiplicative", sp=12)),
    ("damped", ExponentialSmoothing(trend="add", damped=True, seasonal="multiplicative", sp=12))
])

forecaster.fit(y_train,fh=np.arange(1)+1)

In [ ]:
fh = np.arange(len(y_test)) + 1
predict = forecaster._predict(fh)

In [ ]:
plot_ys(predict,y_test,labels=['predict','actual'])
plt.plot()

In [ ]:
np.sum(se(predict.to_numpy(),y_test.to_numpy()))

In [ ]:
smape_loss(predict[1:],y_test[1:])

## Normal Hedge Algorithm Ensemble

In [ ]:
loss_func = lambda x,y: se(x/600,y/600)
# hedge_expert = HedgeExpertEnsemble(3,80,OnlineHedgeDoubling,loss_func=loss_func)
hedge_expert = HedgeExpertEnsemble(3,80,NormalHedge,loss_func=loss_func)

In [ ]:
forecaster = OnlineEnsembleForecaster([
    ("ses", ExponentialSmoothing(seasonal="multiplicative", sp=12)),
    ("holt", ExponentialSmoothing(trend="add", damped=False, seasonal="multiplicative", sp=12)),
    ("damped", ExponentialSmoothing(trend="add", damped=True, seasonal="multiplicative", sp=12))
], 
    ensemble_algorithm=hedge_expert)

In [ ]:
forecaster.fit(y_train)
predict = forecaster.update_predict(y_test)

In [ ]:
forecaster.ensemble_algorithm.weights

In [ ]:
plot_ys(predict,y_test,labels=['predict','actual'])
plt.plot()

In [ ]:
np.sum(se(predict.to_numpy(),y_test.to_numpy()[1:]))

In [ ]:
smape_loss(predict,y_test.iloc[1:])

## NNLS Ensemble

In [ ]:
hedge_expert = NNLSEnsemble(3,loss_func=loss_func)

In [ ]:
forecaster = OnlineEnsembleForecaster([
    ("ses", ExponentialSmoothing(seasonal="multiplicative", sp=12)),
    ("holt", ExponentialSmoothing(trend="add", damped=False, seasonal="multiplicative", sp=12)),
    ("damped", ExponentialSmoothing(trend="add", damped=True, seasonal="multiplicative", sp=12))
],
    ensemble_algorithm=hedge_expert)

In [ ]:
forecaster.fit(y_train)

In [ ]:
forecaster._fit_ensemble(y_test)
predict = forecaster.update_predict(y_test)

In [ ]:
plot_ys(predict,y_test,labels=['predict','actual'])
plt.plot()
forecaster.ensemble_algorithm.weights

In [ ]:
np.sum(se(predict.to_numpy(),y_test.to_numpy()[1:]))

In [ ]:
smape_loss(predict,y_test.iloc[1:])